In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import numpy as np

model_checkpoint = "models/distilbert-ner/checkpoint-213"
save_directory = "../onnx/"

# Load a model from transformers and export it to ONNX
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/root/miniconda3/envs/training-pipeline/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
dummy_input = tokenizer(text, return_tensors="pt")

In [3]:
torch.onnx.export(
    model, 
    tuple(dummy_input.values()),
    f="../onnx/torch-model.onnx",  
    input_names=['input_ids', 'attention_mask'], 
    output_names=['logits'], 
    dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence'}, 
                  'attention_mask': {0: 'batch_size', 1: 'sequence'}, 
                  'logits': {0: 'batch_size', 1: 'sequence'}}, 
    do_constant_folding=True, 
    opset_version=13, 
)

/root/miniconda3/envs/training-pipeline/lib/python3.10/site-packages/transformers/models/distilbert/modeling_distilbert.py:230: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


In [5]:
import onnx
model_onnx = onnx.load("../onnx/torch-model.onnx")
onnx.checker.check_model(model_onnx)

In [6]:
text2 = "My name is Tom, I live in New York and my girlfriend's name is Elaine. Our parents live in Viet Nam, Nha Trang city, and their names are Que and Mai"
text2 = tokenizer(text2, return_tensors='pt')

In [7]:
import onnxruntime as ort
import numpy as np
ort_sess = ort.InferenceSession('../onnx/torch-model.onnx')
outputs = ort_sess.run(None, {'input_ids': text2['input_ids'].numpy(),
                            'attention_mask':  text2['attention_mask'].numpy()})

label = np.argmax(outputs, axis=3)
label

array([[[0, 0, 0, 0, 9, 0, 0, 0, 0, 7, 7, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0,
         0, 0, 0, 7, 7, 0, 7, 0, 7, 7, 7, 0, 0, 0, 0, 0, 9, 0, 9, 0]]])

In [8]:
import bentoml

signatures = {
    "run": {"batchable": True},
}
bentoml.onnx.save_model("onnx_ner", model_onnx, signatures=signatures)

Model(tag="onnx_ner:4otq4vsmcswu4asc", path="/root/bentoml/models/onnx_ner/4otq4vsmcswu4asc/")